In [ ]:
!pip install langchain
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install sentence_transformers

In [ ]:
!pip install unstructured

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

# Extract data from url

In [ ]:
import nltk
urls = [
    "https://ineuron.ai/"
]

In [ ]:
print(nltk.__version__)

In [ ]:
# import nltk
# nltk.download('punkt_tab')

In [ ]:
loader=UnstructuredURLLoader(urls=urls)
data = loader.load()

In [ ]:
data

# Chunkings 

In [ ]:
text_splitter = CharacterTextSplitter(separator='\n', chunk_size = 1000, chunk_overlap=200)

In [ ]:
text_chunk = text_splitter.split_documents(data)

In [ ]:
len(text_chunk)

In [ ]:
text_chunk[0]

# Embedding model 

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

In [ ]:
query_result

# Connecting to Pinecode client

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
!pip install pinecone-client

In [ ]:
!pip install -qU pinecone-client==3.1.0 pandas==2.0.3

In [ ]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY')

# configure client
pc = Pinecone(api_key=api_key)



In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
# Giving our index a name
index_name = "test"

In [ ]:
from langchain.vectorstores import Pinecone

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunk], embeddings, index_name=index_name)

In [ ]:
# Create LLM wrapper

In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", 
                                          use_auth_token=True)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    device_map='cpu',
    torch_dtype=torch.float16,  # You can remove this line as well if needed
    use_auth_token=True
)


In [ ]:
pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map=-1,# Use 0 for GPU, or -1 for CPU
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
# llm.predict("Please provide a concise summary of the Book Harry Potter")

# Initialize the Retrieval QA

In [ ]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())
query = "Tell me the course price of Full Stack Data Science with Generative AI provide by ineuron"



In [ ]:
print(qa.run(query))